In [91]:
# Import necessary packages and libraries
import pandas as pd
import json
from PIL import Image, ImageDraw, ImageFont
from PIL.ImageOps import scale
import math
from itertools import groupby
import numpy as np
from statistics import mean
import datetime

In [92]:
class BackgroundTemplate:
    '''
    Description: Background Template helps to define box methods to create the graphical abstract template 
    which will house all the necessary information needed for the graphical abstract visualizer
    '''
    def __init__(self, image_size, background_color='white'):
        self.image = Image.new('RGB', image_size, color=background_color)
        self.draw = ImageDraw.Draw(self.image)
        self.boxes = []

    def add_box(self, top_left, bottom_right, outline_color='black', fill_color=None, text=None, text_color='black'):
        box = {'top_left': top_left, 'bottom_right': bottom_right, 'outline_color': outline_color, 'fill_color': fill_color, 'text': text, 'text_color': text_color}
        self.boxes.append(box)

    def draw_boxes(self):
        for box in self.boxes:
            self.draw.rectangle([box['top_left'], box['bottom_right']], outline=box['outline_color'], fill=box['fill_color'])
            if box['text']:
                self._draw_centered_text(box['text'], box['top_left'], box['bottom_right'], box['text_color'])

    def _draw_centered_text(self, text, top_left, bottom_right, text_color, font = None):
        if font == None:
            font_path = "./fonts/Aptos-Bold.ttf"
            font_size = 26
            font = ImageFont.truetype(font_path, font_size)
        bbox = self.draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        box_width = bottom_right[0] - top_left[0]
        box_height = bottom_right[1] - top_left[1]
        x = top_left[0] + (box_width - text_width) / 2
        y = top_left[1] + (box_height - text_height) / 2
        self.draw.text((x, y), text, fill=text_color, font=font)


    def get_image(self):
        return self.image

    def paste(self, icon, icon_position, icon_type):
        self.image.paste(icon, icon_position, icon_type)

    def add_enclosed_boxes(self, enclosing_box_top_left, enclosing_box_bottom_right, num_inner_boxes, spacing, colors, texts):
        # Define the enclosing box
        self.add_box(enclosing_box_top_left, enclosing_box_bottom_right, outline_color='black', fill_color=None)
        
        # Calculate inner box dimensions and spacing
        enclosing_width = enclosing_box_bottom_right[0] - enclosing_box_top_left[0]
        enclosing_height = enclosing_box_bottom_right[1] - enclosing_box_top_left[1]
        box_width = enclosing_width - 2 * spacing
        box_height = (enclosing_height - (num_inner_boxes + 1) * spacing) // num_inner_boxes
        
        # Add inner boxes
        for i in range(num_inner_boxes):
            top_left = (enclosing_box_top_left[0] + spacing, enclosing_box_top_left[1] + spacing + i * (box_height + spacing))
            bottom_right = (top_left[0] + box_width, top_left[1] + box_height)
            self.add_box(top_left, bottom_right, outline_color='black', fill_color=colors[i % len(colors)], text=texts[i], text_color='white')
    
    def draw_dashed_line(self, start, end, dash_length, gap_length, color, width):
        """
        Draws a dashed line between two points.
        """
        x1, y1 = start
        x2, y2 = end
        
        # Calculate total line length
        total_length = ((x2 - x1)**2 + (y2 - y1)**2)**0.5
    
        # Calculate the direction vector (unit vector)
        dx = (x2 - x1) / total_length
        dy = (y2 - y1) / total_length
    
        # Draw dashes
        current_pos = 0
        while current_pos < total_length:
            # Start point of the dash
            dash_start = (x1 + dx * current_pos, y1 + dy * current_pos)
            
            # End point of the dash (clamped to the total length)
            dash_end = (
                x1 + dx * min(current_pos + dash_length, total_length),
                y1 + dy * min(current_pos + dash_length, total_length)
            )
            
            # Draw the dash
            self.draw.line([dash_start, dash_end], fill=color, width=width)
            
            # Move the position forward by dash + gap
            current_pos += dash_length + gap_length


In [93]:
def add_labels_logic(data):
        
    '''
        Description: Labels the age of mice during the treatment procedure from initial age to 
        final termination. This function dynamically scales the number of weeks in or out based on the 
        largest termination date present in the dataset. 
        
    '''

    DOB = []
    Irrad_Date = []
    Termination_Date_Initial = []
    Termination_Date_Final = []
    return_dict = {}

    if not data['DOB'].isnull().any():
        DOB = data['DOB'].unique().tolist()
        DOB = [data.date() for data in DOB]
        DOB = sorted(DOB)
        

    if not data['Irradiation Date'].isnull().any():
        Irrad_Date = data['Irradiation Date'].unique().tolist()
        Irrad_Date = [data.date() for data in Irrad_Date]
        Irrad_Date = sorted(Irrad_Date)

    
    if not data['Termination Date Initial'].isnull().any():
        Termination_Date_Initial = data['Termination Date Initial'].unique().tolist()
        Termination_Date_Initial = [data.date() for data in Termination_Date_Initial]
        Termination_Date_Initial = sorted(Termination_Date_Initial)
    
    if not data['Termination Date Final'].isnull().any():
        Termination_Date_Final = data['Termination Date Final'].unique().tolist()
        Termination_Date_Final = [data.date() for data in Termination_Date_Final]
        Termination_Date_Final = sorted(Termination_Date_Final)
        
        # print(f"DOB: {DOB}")
        return_dict['DOB'] = DOB

        # print(f"Irradiation Dates: {Irrad_Date}")
        return_dict['Irrad_Date'] = Irrad_Date

        # print(f"Termination Date Initial: {Termination_Date_Initial}")
        return_dict['Termination_Date_Initial'] = Termination_Date_Initial

        # print(f"Termination Date Final: {Termination_Date_Final}")
        return_dict['Termination_Date_Final'] = Termination_Date_Final

        return return_dict

In [94]:
def add_labels(template, return_dict, initial_pos_x, initial_pos_y, final_pos_x, final_pos_y):
    pos_x = initial_pos_x
    pos_y = initial_pos_y
    prev = 0 

    font_path = "./fonts/Aptos-Bold.ttf"
    font_size = 35
    font = ImageFont.truetype(font_path, font_size)
    
    dict_of_weeks = {}

    date_list = return_dict.values()
    try: 
      start_date = return_dict['DOB'][0]
    except:
      start_date = return_dict['Irrad_Date'][0]
    final_date = return_dict['Termination_Date_Final'][len(return_dict['Termination_Date_Final']) - 1]
    
    total_days = int((final_date - start_date).days)

    total_x_delta = int(final_pos_x - initial_pos_x - 100)

    increment_delta = total_x_delta / total_days


    prev = start_date

    all_dates = []
    for values in return_dict.values():
        all_dates.extend(values)
    all_dates = sorted(all_dates)

    for value in all_dates:
      if value == start_date:
        if value in return_dict['DOB']:
          label = '#283618'
          name = "DOB"
        elif value in return_dict['Irrad_Date']:
          label = '#faa307'
          name = "Irradiation Date"
        else:
          name = "Unknown"
        date_obj = datetime.datetime.strptime(str(start_date), "%Y-%m-%d")
        formatted_date = date_obj.strftime("%m/%d/%Y")
        template._draw_centered_text(f"{name}", (pos_x + 10, pos_y - 40), (pos_x + 50, pos_y - 40), text_color = '#283618', font = font)
        template._draw_centered_text(f"{formatted_date}", (pos_x + 10, pos_y + 10), (pos_x + 50, pos_y + 10), text_color = '#283618', font = font)
        dict_of_weeks[start_date] = [pos_x, pos_y]
        template.add_box((pos_x, pos_y + 50), (pos_x + 0.0001, final_pos_y), outline_color= '#C3C3C3', fill_color= None , text=None, text_color='black')
      elif value == final_date:
        # date_obj = datetime.datetime.strptime(str(value), "%Y-%m-%d")
        # formatted_date = date_obj.strftime("%m/%d/%Y")
        # template._draw_centered_text(f"{formatted_date}", (pos_x, pos_y), (pos_x + 20, pos_y), text_color = 'black', font = font)
        # template.add_box((pos_x, pos_y + 20), (pos_x + 0.0001, final_pos_y), outline_color= '#ebedf0', fill_color= None , text=None, text_color='black')
        # dict_of_weeks[start_date] = [pos_x, pos_y]
        break
      elif value == []:
        continue
      else:
        if value in return_dict['DOB']:
          label = '#283618'
          name = "DOB"
        elif value in return_dict['Irrad_Date']:
          label = '#faa307'
          name = "Irradiation Date"
        elif value in return_dict['Termination_Date_Initial']:
          label = '#d00000'
          name = "Date of Death"
        else:
          label = '#9d0208'
          name = "Date of Death"
        
        print(value)
        print(prev)
        delta = int((value - prev).days)
        pos_x += int(delta) * increment_delta
        if delta >= 30:
            date_obj = datetime.datetime.strptime(str(value), "%Y-%m-%d")
            formatted_date = date_obj.strftime("%m/%d/%Y")
            template._draw_centered_text(f"{name}", (pos_x + 10, pos_y - 40), (pos_x + 20, pos_y - 40), text_color = label, font = font)
            template._draw_centered_text(f"{formatted_date}", (pos_x + 10, pos_y + 10), (pos_x + 20, pos_y + 10), text_color = label, font = font)
            template.add_box((pos_x, pos_y + 50), (pos_x + 0.0001, final_pos_y), outline_color= '#C3C3C3', fill_color= None , text=None, text_color='black')
        dict_of_weeks[value] = [pos_x, pos_y, label]
        
        prev = value

    print("Finished")
    return dict_of_weeks




In [95]:
df_dict = pd.read_excel('Comparisons.xlsx', sheet_name = None)
df2 =  pd.read_excel('Experiment Info.xlsx')
df_dict.keys()

dict_keys(['G380-11', 'Dynan Si-28', 'Dynan GCRSIM'])

In [96]:
df_list = []
for sheet in df_dict.keys():
    print(sheet)
    df = df_dict[sheet]
    df_list.append(df)

G380-11
Dynan Si-28
Dynan GCRSIM


In [97]:
df_list[0]

,Group,Beam,Dose,Subjects,DOB,Irradiation Date,Termination Date Initial,Termination Date Final,Gender,Treatment,Order
0,1,260 MeV/u Silicon,4,132,2010-03-01,2010-04-01,2011-07-23,2011-08-03,Female,NaN,1
1,2,260 MeV/u Silicon,8,114,2010-03-01,2010-04-01,2011-07-23,2011-08-03,Female,NaN,1
2,3,Control 1,0,24,2010-03-01,2010-04-01,2011-07-23,2011-08-03,Female,NaN,1
3,4,260 MeV/u Silicon,16,74,2010-03-01,2010-04-01,2011-07-23,2011-08-03,Female,NaN,1
4,5,260 MeV/u Silicon,32,49,2010-03-01,2010-04-01,2011-07-23,2011-08-03,Female,NaN,1
5,6,Control 2,0,29,2010-03-01,2010-04-01,2011-07-23,2011-08-03,Female,NaN,1


In [98]:
df_list[1]

,Group,Beam,Dose,Subjects,DOB,Irradiation Date,Termination Date Initial,Termination Date Final,Gender,Treatment,Order,Termination Before Scheduled Endpoint
0,A1,Silicon,0.40,5,2018-11-06,2019-04-25,2019-05-22,2019-05-22,Male,Negative,1,NaN
1,A2,Silicon,0.40,5,2018-11-06,2019-04-25,2019-05-22,2019-05-22,Female,Negative,1,NaN
2,A3,Silicon,0.40,5,2018-11-06,2019-04-25,2019-10-22,2019-10-22,Male,Negative,3,NaN
3,A4,Silicon,0.40,5,2018-11-06,2019-04-25,2019-10-22,2019-10-22,Female,Negative,3,NaN
4,C1,Silicon,0.40,6,2018-11-06,2019-04-25,2019-05-23,2019-05-23,Male,Positive,1,NaN
5,C2,Silicon,0.40,4,2018-11-06,2019-04-25,2019-05-23,2019-05-23,Female,Positive,1,NaN
6,C3,Silicon,0.40,5,2018-11-06,2019-04-25,2019-10-23,2019-10-23,Male,Positive,3,NaN
7,C4,Silicon,0.40,5,2018-11-06,2019-04-25,2019-10-23,2019-10-23,Female,Positive,3,NaN
8,K1,Controls,0.00,5,2018-05-22,2018-11-04,2018-12-11,2018-12-11,Male,Negative,2,NaN
9,K2,Controls,0.00,5,2018-05-22,2018-11-09,2018-12-10,2018-12-10,Female,Negative,2,NaN


In [99]:
 def draw_arrow(template, data_row, dict_of_weeks, pos_x, pos_y):
    font_path = "./fonts/Aptos-Bold.ttf"
    font_size = 35
    font = ImageFont.truetype(font_path, font_size)

    font_large = ImageFont.truetype(font_path, 2.5 * 30)

    font_small = ImageFont.truetype(font_path, 30)

    lineweight = 4
    if data_row['DOB'] != None:
        try:
            dob_pos_list = dict_of_weeks.get(data_row['DOB'].date()) 
        except: 
            pass

    if data_row['Irradiation Date'] != None:
        init_pos_list = dict_of_weeks.get(data_row['Irradiation Date'].date())
        if data_row['Beam'] != None and data_row['Dose'] != None and data_row['Group'] != None:
            try:
                
                beam = data_row['Beam']
                dose = data_row['Dose']
                group = data_row['Group']
                template._draw_centered_text(f"{group} | {beam} | {dose} cGy", (dob_pos_list[0], dob_pos_list[1] + pos_y - 30), (init_pos_list[0], init_pos_list[1] + pos_y - 30), text_color = 'black', font = font)
            except:
                pass       
        template.add_box((init_pos_list[0] - 1, init_pos_list[1] + pos_y - lineweight * 10), (init_pos_list[0] + 1, init_pos_list[1] + pos_y + lineweight * 10), outline_color= None, fill_color='#faa307')
        try:
            template.draw_dashed_line((dob_pos_list[0], dob_pos_list[1] + pos_y), (init_pos_list[0], init_pos_list[1] + pos_y), 10, 10, '#4299B3', lineweight)
        except:
            pass
        
    
        if data_row['Termination Date Initial'] != None:
            termination_pos_list = dict_of_weeks.get(data_row['Termination Date Initial'].date())
            subjects = data_row['Subjects']
            template._draw_centered_text(f"{subjects} subjects", (init_pos_list[0], termination_pos_list[1] + pos_y - 40), (termination_pos_list[0], termination_pos_list[1] + pos_y - 40), text_color = 'black', font = font_small)
            if subjects > 5:
                lineweight = lineweight * 2
            elif subjects > 10:
                lineweight = lineweight * 3
            elif subjects > 20:
                lineweight = lineweight * 4
            elif subjects > 50:
                lineweight = lineweight * 5
            
            if data_row['Gender'] != None:
                gender = str(data_row['Gender'])
                if gender == "Male":
                    fill = "#1E88E5"
                else:
                    fill = "#F52797"
                    
            template.draw.line([(init_pos_list[0], init_pos_list[1] + pos_y), (termination_pos_list[0], termination_pos_list[1] + pos_y)], fill = fill, width = lineweight * 3)
            if data_row['Termination Date Final'] != None:
                final_pos_list = dict_of_weeks.get(data_row['Termination Date Final'].date())
                if final_pos_list != None:
                    template.draw.line([(termination_pos_list[0], termination_pos_list[1] + pos_y), (final_pos_list[0], final_pos_list[1] + pos_y)], fill = 'black', width = lineweight)
                    template.add_box((final_pos_list[0] - 1, final_pos_list[1] + pos_y - lineweight * 10), (final_pos_list[0] + 1, final_pos_list[1] + pos_y + lineweight * 10), outline_color= None, fill_color='#9d0208')

            if data_row['Treatment'] != None:
                treatment = str(data_row['Treatment'])
                if treatment == "Positive":
                    fill = "black"
                    if final_pos_list != None:
                        template._draw_centered_text("***", (final_pos_list[0] + 50, final_pos_list[1] + pos_y), (final_pos_list[0] + 80, final_pos_list[1] + pos_y), text_color = fill, font = font_large)
                    else: 
                        template._draw_centered_text("***", (termination_pos_list[0] + 50, termination_pos_list[1] + pos_y), (termination_pos_list[0] + 80, termination_pos_list[1] + pos_y), text_color = fill, font = font_large)
            # template.draw.line([(init_pos_list[0], init_pos_list[1] + pos_y + 10), (termination_pos_list[0], termination_pos_list[1] + pos_y + 10)], fill = fill, width = lineweight)
            template.add_box((termination_pos_list[0] - 1, termination_pos_list[1] + pos_y - lineweight * 10), (termination_pos_list[0] + 1, termination_pos_list[1] + pos_y + lineweight * 10), outline_color= None, fill_color='#d00000')
        
            
   

    return "Arrow Drawn"
        
    

In [100]:
def generate_image(data, sheet_name, final_pos_x, final_pos_y):
        
        # Experiment Initialization
        template = BackgroundTemplate((final_pos_x, final_pos_y))
        
        font_path = "./fonts/Aptos-Bold.ttf"
        font_size = 26
        font = ImageFont.truetype(font_path, font_size)

        # # Define the parameters for the first set of enclosed boxes
        # enclosing_box_top_left1 = (50, 50)
        # enclosing_box_bottom_right1 = (350, 300)
        # num_inner_boxes = 4
        # spacing = 20
        # colors = ['#649de3', '#649de3', '#649de3', '#649de3']
        # texts1 = pi_info

        # # Add the first set of enclosed boxes
        # template.add_enclosed_boxes(enclosing_box_top_left1, enclosing_box_bottom_right1, num_inner_boxes, spacing, colors, texts1)

        # # Define the parameters for the second set of enclosed boxes
        # enclosing_box_top_left2 = (50, 320)
        # enclosing_box_bottom_right2 = (350, 570)
        # num_inner_boxes_2 = 4
        
        # texts2 = exp_info

        # # Add the second set of enclosed boxes
        # template.add_enclosed_boxes(enclosing_box_top_left2, enclosing_box_bottom_right2, num_inner_boxes_2, spacing, colors, texts2)

        # # Define the parameters for the third logo box
        # logo_box_top_left3 = (50, 590)
        # logo_box_bottom_right3 = (350, 650)
        # template.add_box(logo_box_top_left3, logo_box_bottom_right3)

        
        
        
        # Define the parameters for the Treatment Group Visualizer
        # treatment_box_top_left = (370, 50)
        # treatment_box_bottom_right = (final_pos_x - 50, final_pos_y - 50)
        # template.add_box(treatment_box_top_left, treatment_box_bottom_right)

        #Display Weeks
        return_dict = add_labels_logic(data)
        dict_of_weeks = add_labels(template, return_dict, 100, 100, final_pos_x - 100, final_pos_y - 100)

        pos_x = 100
        pos_y = 100
        final_x = final_pos_x - 100
        final_y = final_pos_y - 100
        length = data.shape[0]
        delta_y = (final_y - pos_y) / length

        group_columns_by_instance = input('What is the important group of interest')

        if group_columns_by_instance == 'DOB':
            if not (data['DOB'].isna().all()) and not (data['Treatment'].isna().all()):
                data = data.sort_values(['DOB', 'Treatment', 'Group'])
            elif not (data['DOB'].isna().all()): 
                data = data.sort_values(['DOB', 'Group'])
        elif group_columns_by_instance == 'Recovery':
            if not (data['Order'].isna().all()):
                data = data.sort_values(['Order', 'Group', 'Treatment'])
                
        else:
            print('No Columns passed')
        
        for key, data_row in data.iterrows():
            
            draw_arrow(template, data_row, dict_of_weeks, pos_x, pos_y)
            pos_y += delta_y 

        # Draw all boxes
        template.draw_boxes()
        # Display or save the image
        image_output = template.get_image()
        image_output.show()
        # template.get_image().save(f'{sheet_name}.png')  # Save the image

In [101]:
generate_image(df_list[1], 'Test1', 5000, 3000)

2018-11-04
2018-05-22
2018-11-06
2018-11-04
2018-11-07
2018-11-06
2018-11-09
2018-11-07
2018-11-14
2018-11-09
2018-11-19
2018-11-14
2018-12-06
2018-11-19
2018-12-06
2018-12-06
2018-12-07
2018-12-06
2018-12-07
2018-12-07
2018-12-10
2018-12-07
2018-12-10
2018-12-10
2018-12-11
2018-12-10
2018-12-11
2018-12-11
2019-04-25
2018-12-11
2019-04-26
2019-04-25
2019-05-13
2019-04-26
2019-05-13
2019-05-13
2019-05-14
2019-05-13
2019-05-14
2019-05-14
2019-05-15
2019-05-14
2019-05-15
2019-05-15
2019-05-16
2019-05-15
2019-05-16
2019-05-16
2019-05-17
2019-05-16
2019-05-20
2019-05-17
2019-05-22
2019-05-20
2019-05-22
2019-05-22
2019-05-23
2019-05-22
2019-05-23
2019-05-23
2019-05-29
2019-05-23
2019-05-29
2019-05-29
2019-10-18
2019-05-29
2019-10-18
2019-10-18
2019-10-22
2019-10-18
2019-10-22
2019-10-22
2019-10-23
2019-10-22
2019-10-23
2019-10-23
2020-04-27
2019-10-23
2020-05-04
2020-04-27
2020-05-11
2020-05-04
2020-10-23
2020-05-11
2020-10-28
2020-10-23
2020-11-02
2020-10-28
2020-11-03
2020-11-02
2020-11-03

What is the important group of interest Recovery


In [55]:
generate_image(df_list[0], 'Test0', 5000, 2000)

2010-04-01
2010-03-01
2011-07-23
2010-04-01
Finished


In [103]:
for sheet in df_dict.keys():
    print(sheet)
    df = df_dict[sheet]
    generate_image(df, 'sheet', 5000, 3000)

G380-11
2010-04-01
2010-03-01
2011-07-23
2010-04-01
Finished


What is the important group of interest Recovery


Dynan Si-28
2018-11-04
2018-05-22
2018-11-06
2018-11-04
2018-11-07
2018-11-06
2018-11-09
2018-11-07
2018-11-14
2018-11-09
2018-11-19
2018-11-14
2018-12-06
2018-11-19
2018-12-06
2018-12-06
2018-12-07
2018-12-06
2018-12-07
2018-12-07
2018-12-10
2018-12-07
2018-12-10
2018-12-10
2018-12-11
2018-12-10
2018-12-11
2018-12-11
2019-04-25
2018-12-11
2019-04-26
2019-04-25
2019-05-13
2019-04-26
2019-05-13
2019-05-13
2019-05-14
2019-05-13
2019-05-14
2019-05-14
2019-05-15
2019-05-14
2019-05-15
2019-05-15
2019-05-16
2019-05-15
2019-05-16
2019-05-16
2019-05-17
2019-05-16
2019-05-20
2019-05-17
2019-05-22
2019-05-20
2019-05-22
2019-05-22
2019-05-23
2019-05-22
2019-05-23
2019-05-23
2019-05-29
2019-05-23
2019-05-29
2019-05-29
2019-10-18
2019-05-29
2019-10-18
2019-10-18
2019-10-22
2019-10-18
2019-10-22
2019-10-22
2019-10-23
2019-10-22
2019-10-23
2019-10-23
2020-04-27
2019-10-23
2020-05-04
2020-04-27
2020-05-11
2020-05-04
2020-10-23
2020-05-11
2020-10-28
2020-10-23
2020-11-02
2020-10-28
2020-11-03
2020-11-0

What is the important group of interest Recovery


Dynan GCRSIM
2018-11-04
2018-05-22
2018-11-06
2018-11-04
2018-11-09
2018-11-06
2018-11-14
2018-11-09
2018-11-19
2018-11-14
2018-12-10
2018-11-19
2018-12-10
2018-12-10
2018-12-11
2018-12-10
2018-12-11
2018-12-11
2019-04-26
2018-12-11
2019-05-13
2019-04-26
2019-05-13
2019-05-13
2019-05-16
2019-05-13
2019-05-16
2019-05-16
2019-05-21
2019-05-16
2019-05-24
2019-05-21
2019-05-24
2019-05-24
2019-05-28
2019-05-24
2019-05-28
2019-05-28
2019-05-29
2019-05-28
2019-05-29
2019-05-29
2019-10-18
2019-05-29
2019-10-18
2019-10-18
2019-10-24
2019-10-18
2019-10-24
2019-10-24
2019-10-25
2019-10-24
2019-10-25
2019-10-25
2019-11-02
2019-10-25
2019-12-02
2019-11-02
2019-12-02
2019-12-02
2019-12-03
2019-12-02
2019-12-03
2019-12-03
2020-05-04
2019-12-03
2020-05-04
2020-05-04
2020-05-05
2020-05-04
2020-05-05
2020-05-05
2020-10-23
2020-05-05
2020-11-03
2020-10-23
2020-11-05
2020-11-03
2020-11-10
2020-11-05
2020-11-12
2020-11-10
2021-04-22
2020-11-12
2021-04-28
2021-04-22
2021-04-28
2021-04-28
Finished


What is the important group of interest Recovery
